In [ ]:
# pip install scikit-learn

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import pandas as pd
import torch
import numpy as np
from sklearn.metrics import classification_report, hamming_loss, accuracy_score
from sklearn.metrics import f1_score


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
df = pd.read_csv("/content/drive/MyDrive/chexpert_data_v2/valid.csv", sep=",")

In [ ]:
image_embedding_path = "/content/drive/MyDrive/Embedings/Image_embeding/image_embeddings_LRF30%.pt"
text_embedding_path = "/content/drive/MyDrive/Embedings/Text_embeding/disease_text_embeddings.pt"

In [ ]:
image_embeddings = torch.load(image_embedding_path)
text_embeddings = torch.load(text_embedding_path)

In [ ]:
disease_columns = list(text_embeddings.keys())

In [ ]:
disease_columns

In [ ]:
print(df.columns)


In [ ]:
disease_columns = [
    'Atelectasis',
    'Cardiomegaly',
    'Consolidation',
    'Edema',
    'Pleural Effusion',
    'Pneumonia',
    'Pneumothorax'
]




In [ ]:
label_dict = {}
prefix = "CheXpert-v1.0-small/"

for i, row in df.iterrows():
    path = row["Path"]

    if path.startswith(prefix):
        path = path[len(prefix):]

    label = []
    for disease in disease_columns:
        val = row[disease]
        if pd.isna(val):
            label.append(0.0)
        else:
            label.append(float(val))
    label_dict[path] = torch.tensor(label)


In [ ]:
img_paths = list(image_embeddings.keys())


In [ ]:
# img_paths

In [ ]:
labels = torch.stack([label_dict[p] for p in img_paths])


In [ ]:
labels

In [ ]:
img_keys = list(image_embeddings.keys())
img_tensor = torch.stack([image_embeddings[k] for k in img_keys])  # [N, r]

disease_keys = list(text_embeddings.keys())
text_tensor = torch.stack([text_embeddings[k] for k in disease_keys])  # [C, r]


In [ ]:
print(img_tensor.size())
print(text_tensor.size())


In [ ]:
image_repr = img_tensor.mean(dim=1)
text_tensor = torch.stack([text_embeddings[k] for k in text_embeddings]).to(device)  # [7, 768]
text_tensor = F.normalize(text_tensor, dim=1)


In [ ]:
proj = torch.nn.Linear(64, 768).to(device)
image_proj = proj(image_repr)  # [234, 768]


In [ ]:
image_proj = F.normalize(image_proj, dim=1)
text_tensor = F.normalize(text_tensor, dim=1)


In [ ]:
similarity = image_proj @ text_tensor.T  # [234, 7]


In [ ]:
print(image_repr.size())

In [ ]:
optimizer = torch.optim.Adam(proj.parameters(), lr=1e-4)

In [ ]:
# labels = labels.to(device)

In [ ]:
probs = torch.sigmoid(similarity)  # [234, 7]
preds = (probs > 0.5).int()


In [ ]:
threshold = 0.5
preds = (probs > threshold).float()


In [ ]:
preds

In [ ]:
accuracy = (preds == labels).all(dim=1).float().mean()
print(f"Exact match accuracy: {accuracy.item():.4f}")


In [ ]:
sample_accuracy = (preds == labels).float().mean()
print(f"Sample-wise accuracy: {sample_accuracy.item():.4f}")


In [ ]:
per_label_acc = ((preds == labels).float().mean(dim=0))
for i, disease in enumerate(disease_columns):
    print(f"{disease}: {per_label_acc[i].item():.4f}")

macro_accuracy = per_label_acc.mean()
print(f"Macro (mean per-label) accuracy: {macro_accuracy.item():.4f}")


In [ ]:
y_true = labels.cpu().numpy()
y_pred = preds.cpu().numpy()

target_names = ['Atelectasis', 'Cardiomegaly', 'Consolidation', 'Edema',
                'Pleural Effusion', 'Pneumonia', 'Pneumothorax']

print("🔍 Classification Report (Macro & Per-label):")
print(classification_report(y_true, y_pred, target_names=target_names, zero_division=0))

#  Hamming Loss
hl = hamming_loss(y_true, y_pred)
print(f"\n Hamming Loss: {hl:.4f}")

# ⚖️ Exact Match Ratio (sample-wise accuracy)
exact_match = np.all(y_true == y_pred, axis=1).mean()
print(f" Exact Match Accuracy: {exact_match:.4f}")

In [ ]:
best_threshold = 0.0
best_f1 = 0.0

for t in np.arange(0.1, 0.9, 0.05):
    preds = (probs > t).int()
    f1 = f1_score(y_true, preds.numpy(), average='macro', zero_division=0)
    if f1 > best_f1:
        best_f1 = f1
        best_threshold = t

print(f"  threshold: {best_threshold:.2f} با F1: {best_f1:.4f}")


In [63]:
text_tensor.shape

torch.Size([7, 768])

In [64]:
labels.shape

torch.Size([234, 7])

In [62]:
image_proj.shape

torch.Size([234, 768])

In [66]:
image_proj = F.normalize(image_proj, dim=1)
text_tensor = F.normalize(text_tensor, dim=1)


In [68]:
similarity = image_proj @ text_tensor.T


In [69]:
loss = F.binary_cross_entropy_with_logits(similarity, labels.float())


In [70]:
optimizer.zero_grad()
loss.backward()
optimizer.step()

In [71]:
# for imgs, labels in dataloader:  # imgs: [B, C, H, W] / labels: [B, num_classes]
#     imgs = imgs.to(device)
#     labels = labels.to(device)

#     image_proj = image_encoder(imgs)             # [B, D]
#     image_proj = F.normalize(image_proj, dim=1)


#     text_proj = F.normalize(text_tensor, dim=1)

#     similarity = image_proj @ text_proj.T        # [B, num_classes]
#     loss = F.binary_cross_entropy_with_logits(similarity, labels.float())

#     optimizer.zero_grad()
#     loss.backward()
#     optimizer.step()
